# 실습 04: OCR 엔진 성능 비교

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab04_OCR엔진비교.ipynb)

## 🎯 학습 목표
- 여러 OCR 엔진 비교 (Tesseract, PaddleOCR, EasyOCR)
- 정확도, 속도, 신뢰도 측정
- 최적 OCR 엔진 선택 방법

## ⏱️ 소요 시간: 30분
## 📊 난이도: ⭐⭐⭐☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
# --force-reinstall: 캐시 무시하고 최신 버전 강제 재설치
!pip install -q --force-reinstall --no-cache-dir git+https://github.com/leecks1119/document_ai_lecture.git
!apt-get install -y tesseract-ocr tesseract-ocr-kor


In [ ]:
# OCRBenchmark 클래스 사용
from docai_course.ocr import OCRBenchmark
from PIL import Image, ImageDraw
import numpy as np

# OCR 벤치마크 객체 생성 (GPU는 자동 감지)
benchmark = OCRBenchmark()

# 샘플 이미지 생성
img = Image.new('RGB', (600, 200), color='white')
draw = ImageDraw.Draw(img)
draw.text((20, 20), "삼성전자 주식회사\n계약서\n2025년 1월 15일\n담당자: 홍길동\n연락처: 02-1234-5678", fill='black')
img.save('sample_document.jpg')

print("✅ 샘플 이미지 생성 완료!")


In [ ]:
# OCR 엔진 비교 실행
ground_truth = "삼성전자 주식회사 계약서 2025년 1월 15일 담당자: 홍길동 연락처: 02-1234-5678"

results = benchmark.run_comparison(
    image_path='sample_document.jpg',
    ground_truth=ground_truth
)

print("\n✅ OCR 비교 완료!")


In [ ]:
# 결과 분석 및 시각화
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame([r for r in results if 'error' not in r])

print("\n📊 OCR 엔진 비교 결과:")
print(df[['engine', 'character_accuracy', 'processing_time', 'confidence']])

# 최고 성능 엔진
best = df.loc[df['character_accuracy'].idxmax()]
print(f"\n🏆 최고 정확도: {best['engine']} ({best['character_accuracy']:.2f}%)")

# 결과 저장
benchmark.save_results(results, 'lab04_results.json')
print("\n✅ 결과 저장 완료!")
